# Leo - Accumulations
## Beachten
* RnD ist null, **wird aber exportiert**. Falls nächstes Mal nicht Null, überprüfen, ob Script RnD korrekt übernimmt


In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("30.06.2020 Switzerland_Scienceindustries_German.pdf", pages='all', lattice=True)

## Format Table

In [3]:
df_export = df.copy()

#Shift
column = df_export.columns[0]
df_export[df_export[column].notna()] = df_export[df_export[column].notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[[0, 8]],axis=1)
df_export = df_export.drop(df.columns[10:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]


# Select
amount = df_export[df_export.type.str.contains('Gesamtbetrag entfallend auf die Wertetransfers', na=False)]
recipients = df_export[df_export.type.str.contains('Anzahl Empfänger in Offenlegung', na=False)]
rnd = df_export[df_export.type.str.contains('Geldwerte Leistungen im Zusammenhang mit Forschung', na=False)]

df_export = pd.concat([amount, recipients, rnd])

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Rename first column
df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Shift RND from Sponsorship to total
df_export.loc[df_export.type == 'rnd', 'total'] = df_export.sponsorship
df_export.loc[df_export.type == 'rnd', "sponsorship"] = np.NaN

export_acumulations(df_export, 'leo')

saved


In [4]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')

In [4]:
df_export

,type,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,source
0,hcp_amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leo
1,hco_amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leo
2,hcp_count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leo
3,hco_count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,leo
4,rnd,0.0,NaN,NaN,NaN,NaN,NaN,896421.51,leo
